# 자소서 질문 문항 분류 모델 설계

## 데이터 준비


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive



### json파일 데이터 설명
1. 5834 건의 합격자 자소서 데이터
2. 건당 평균 4 개의 질문 문항이 있음
3. tree구조를 가지고 있으며 그 내용은 아래와 같음.

'000000'(pk) : 6자리 자소서 id번호  
하위목록으로 'resume_info', 'grade', 'qus_ans', 'advice'를 가짐.  
resume_info : 자소서마다 resume_info의 하위목록의 개수가 다르며 다양함.  
grade(int) : 자소서마다 전문가의 평점(5점만점)  

qus_ans : qus_ans로 접근하면 질답 번호 '0', '1', '2', '3' 이 key 값으로 나오고, 해당 key값으로 접근하면, dictionary로 구성된 질문(key)과 답변(value)이 호출된다.  
advice(str) : 전문가의 총평이 있거나 없기도 함.  
resume_info: 자소서 작성자의 스펙 및 지원분야, 지원 회사, 조회수

In [4]:
import json

path = 'drive/My Drive/tabditor/수집 데이터/잡코리아 합격자 자소서/jasoseo_merged_000001_005834.json'

with open(path,'r') as f:
    str_json = f.read()


json_data = json.loads(str_json)

json_data

Output hidden; open in https://colab.research.google.com to view.

In [8]:
len(json_data)

5834

json 파일에서 자소서 질문문항 가져오기

In [0]:
title_list=[]

for i in range(len(json_data)):
    for k,v in json_data['{:06d}'.format(i)]['qus_ans'].items():
        title_list.append(list(v.keys())[0])
    

In [37]:
title_list[:10]

['우리공사 및 지원한 직무에 관심이 생긴 계기를 기술하고, 우리공사에서 근무하면서 어떤 부분을 성취하고 싶은지 기술해 주세요. (1000자)',
 '귀하가 스스로 문제를 발견하고, 이를 해결했던 경험에 대해 기술해 주세요.(문제발견 계기 및 해결과정, 해결방법 선택이유, 결과를 포함하여 기술)',
 '의사소통 과정에서 발생한 오해를 풀기 위해 노력을 기울였던 경험에 대해 기술해주세요.(당시 상황과 오해의 원인, 해결 노력, 결과를 포함하여 기술)',
 '지원 분야와 관련된 우리공사의 주요 사업 중 본인이 어떤 부분에 기여하고 싶은지 기술해 주십시오.(본인의 직무역량 및 강점을 중심으로 기술하며, 직무역량 및 강점이 잘 나타난 사례를 포함하여 기술)',
 '우리 회사에 지원한 동기에 대해 기술해주세요',
 '스포츠에 대한 본인의 관심도에 대해 자유롭게 기술해 주세요',
 '지금까지의 경험 중 해결하기 힘든 문제에 부딪혀 어려움을 겪었으나 이를 극복한 사례가 있다면 기술해 주세요',
 '본인 성격의 장단점에 대해 이야기 해주세요',
 'gs리테일에 지원하게 된 동기',
 '지금까지 가장 큰 성공과 큰 실패']

In [0]:
import pandas as pd

df = pd.DataFrame(title_list,columns=['question'])

In [44]:
df

,question
0,"우리공사 및 지원한 직무에 관심이 생긴 계기를 기술하고, 우리공사에서 근무하면서 어..."
1,"귀하가 스스로 문제를 발견하고, 이를 해결했던 경험에 대해 기술해 주세요.(문제발견..."
2,의사소통 과정에서 발생한 오해를 풀기 위해 노력을 기울였던 경험에 대해 기술해주세요...
3,지원 분야와 관련된 우리공사의 주요 사업 중 본인이 어떤 부분에 기여하고 싶은지 기...
4,우리 회사에 지원한 동기에 대해 기술해주세요
...,...
23424,"타인과의 협업 과정에서 나타나는 본인의 장점 및 단점, 단점을 극복하기 위한 노력 ..."
23425,자기자신을 잘 표현할 수 있는 경험과 사례를 자유롭게 기술하시오. (1000자)
23426,샘표식품㈜에 지원하는 구체적인 이유와 샘표식품에서 여러분을 선택해야 하는 이유에 대...
23427,"지원하신 직무에 자신이 적합한 이유를 설명하여 주십시오. (경험, 준비, 노력, 역..."


## 질문문항 분류하고 labeling 하기

질문문항 

0	성장과정  
1	성격 장단점, 생활신조 / 윤리의식(환경, 직업윤리)  
2	직무역량 (강점만 어필하는 느낌) / 강약점 (직무상의 강약점, 학과, 자격증, 수강과목 따라서 이런일에 역량을 발휘할 수 있습니다.)  
3	지원동기 및 입사후 포부 / 입사를 위한 준비과정 / 회사 이해도 (어떤 사업을 하고 있는지, 혁신 (입사 후 제안 서비스))  
4	사회경험 (인턴, 동아리, 공모전, 봉사활동, 아르바이트 및 학창시절) / 도전, 성취 경험 /   
5	경력사항 (일을 했던 경력)  
6	사회이슈 (사회의 특정 사건에 대해서 어떤 견해를 가지고 있는지)  
7	조직적응력, 사회성 (대인관계) > 팀워크 / 소통 (이견있을 때 어떻게 해결? / 리더십)  
8	문제해결력 (의사결정-상사가 지시한 일/ 시한이 급한 일 어떤 일에 우선순위를 둘 것인가, 시나리오 문제), 정보능력 (잘 안나올 것같음)  


In [0]:
str_explain_category = '''0	성장과정
1	성격 장단점, 생활신조 / 윤리의식(환경, 직업윤리)
2	직무역량(강점만 어필하는 느낌) / 강약점(직무상의 강약점, 학과, 자격증, 수강과목 따라서 이런일에 역량을 발휘할 수 있습니다.)
3	지원동기 및 입사후 포부 / 입사를 위한 준비과정 / 회사 이해도(어떤 사업을 하고 있는지, 혁신(입사 후 제안 서비스))
4	사회경험(인턴, 동아리, 공모전, 봉사활동, 아르바이트 및 학창시절) / 도전, 성취 경험 / 
5	경력사항(일을 했던 경력)
6	사회이슈(사회의 특정 사건에 대해서 어떤 견해를 가지고 있는지)
7	조직적응력, 사회성(대인관계) > 팀워크 / 소통(이견있을 때 어떻게 해결? / 리더십)
8	문제해결력(의사결정-상사가 지시한 일/ 시한이 급한 일 어떤 일에 우선순위를 둘 것인가, 시나리오 문제), 정보능력(잘 안나올 것같음)
'''

In [0]:
import random
import numpy as np

## func_int2onehot() 함수 결과 예시
# func_int2onehot([0, 1, 3])    # 0번 label, 1번 label, 3번 label로  구성된 정수형 벡터
# ->> [1, 1, 0, 1]              # label을 index로 변환해서 multi-label 형식으로 변환
                                

def func_int2onehot(var_int_vec):
    ndim = 9    # 질문 문항의 분류 개수 (9가지)
    res_vec = np.zeros(ndim)
    for element in var_int_vec:
        res_vec[element] = 1
    
    return res_vec

l_qus_label= list()
l_str_question = list()
l_int_idx = list()
labeled_data= pd.DataFrame()

def category_classification(question_data):
    
    nb_qus = 20   # 몇 문항 할 건지 선택
    idx_s = 0    # 시작할 문항 번호 선택, idx_s 는 0 부터 시작한다. 
    idx_e = idx_s + nb_qus

    for idx in range(idx_s, idx_e):
        str_question = question_data['question'][idx]
        # 질문번호와 질문 문항을 출력한다.
        print(idx, str_question)
        

        # print('#0: 성장과정	#1:성격/업무상 장단점	#2:사회경험, 학창시절	#3:지원동기 및 입사포부')    
        print(str_explain_category)
        label = input('분류를 입력하시오\n')
        if label == '#':    # 탈출문
            break
        # ex) 0,1,3

        
        converted_label = np.array(label.split(',')).astype(int)
        
        print(converted_label) # [0, 1,  3]
        onehot_vec = func_int2onehot(converted_label)    # [1, 1, 0, 1]
        qus_label = str(onehot_vec)

        l_str_question.append(str_question)
        l_int_idx.append(idx)
        l_qus_label.append(qus_label)


    labeled_data['idx'] =l_int_idx
    labeled_data['label'] = l_qus_label
    labeled_data['qus'] = l_str_question

In [164]:
category_classification(df)

0 우리공사 및 지원한 직무에 관심이 생긴 계기를 기술하고, 우리공사에서 근무하면서 어떤 부분을 성취하고 싶은지 기술해 주세요. (1000자)
0	성장과정
1	성격 장단점, 생활신조 / 윤리의식(환경, 직업윤리)
2	직무역량(강점만 어필하는 느낌) / 강약점(직무상의 강약점, 학과, 자격증, 수강과목 따라서 이런일에 역량을 발휘할 수 있습니다.)
3	지원동기 및 입사후 포부 / 입사를 위한 준비과정 / 회사 이해도(어떤 사업을 하고 있는지, 혁신(입사 후 제안 서비스))
4	사회경험(인턴, 동아리, 공모전, 봉사활동, 아르바이트 및 학창시절) / 도전, 성취 경험 / 
5	경력사항(일을 했던 경력)
6	사회이슈(사회의 특정 사건에 대해서 어떤 견해를 가지고 있는지)
7	조직적응력, 사회성(대인관계) > 팀워크 / 소통(이견있을 때 어떻게 해결? / 리더십)
8	문제해결력(의사결정-상사가 지시한 일/ 시한이 급한 일 어떤 일에 우선순위를 둘 것인가, 시나리오 문제), 정보능력(잘 안나올 것같음)

분류를 입력하시오
3
[3]
1 귀하가 스스로 문제를 발견하고, 이를 해결했던 경험에 대해 기술해 주세요.(문제발견 계기 및 해결과정, 해결방법 선택이유, 결과를 포함하여 기술)
0	성장과정
1	성격 장단점, 생활신조 / 윤리의식(환경, 직업윤리)
2	직무역량(강점만 어필하는 느낌) / 강약점(직무상의 강약점, 학과, 자격증, 수강과목 따라서 이런일에 역량을 발휘할 수 있습니다.)
3	지원동기 및 입사후 포부 / 입사를 위한 준비과정 / 회사 이해도(어떤 사업을 하고 있는지, 혁신(입사 후 제안 서비스))
4	사회경험(인턴, 동아리, 공모전, 봉사활동, 아르바이트 및 학창시절) / 도전, 성취 경험 / 
5	경력사항(일을 했던 경력)
6	사회이슈(사회의 특정 사건에 대해서 어떤 견해를 가지고 있는지)
7	조직적응력, 사회성(대인관계) > 팀워크 / 소통(이견있을 때 어떻게 해결? / 리더십)
8	문제해결력(의사결정-상사가 지시한 일/ 시한이 급한 일 어떤 일에 우선순위

In [165]:
labeled_data

,idx,label,qus
0,0,[0. 0. 0. 1. 0. 0. 0. 0. 0.],"우리공사 및 지원한 직무에 관심이 생긴 계기를 기술하고, 우리공사에서 근무하면서 어..."
1,1,[0. 0. 0. 0. 1. 0. 0. 0. 1.],"귀하가 스스로 문제를 발견하고, 이를 해결했던 경험에 대해 기술해 주세요.(문제발견..."
2,2,[0. 0. 0. 0. 1. 0. 0. 1. 0.],의사소통 과정에서 발생한 오해를 풀기 위해 노력을 기울였던 경험에 대해 기술해주세요...
3,3,[0. 0. 1. 1. 0. 0. 0. 0. 0.],지원 분야와 관련된 우리공사의 주요 사업 중 본인이 어떤 부분에 기여하고 싶은지 기...


In [0]:
labeled_data.to_csv('category_label_9class.csv')